### Кластерный анализ методом DBSCAN

##### Описание данных

Каждая строка набора данных описывает цифру (0 = линия присутствует, 1 = линия отсутствует). Линии соответствуют черточкам на экране калькулятора. 
<br> В данных 7 переменных с именами B-Н:
<br> B - top horizontal
<br> C - upper left vertical
<br> D - upper right vertical
<br> E - middle horizontal
<br> F - lower left vertical
<br> G - lower right vertical
<br> H - bottom horizontal

#### Импорт библиотек

In [2]:
import pandas as pd 
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
import seaborn as sns 
import matplotlib
import matplotlib.pyplot as plt 
%matplotlib inline

#### Загрузка и первичный оосмотр данных

In [87]:
df = pd.read_csv('digit.dat', sep=';')
df

,A,B,C,D,E,F,G,H,A2,B2,C2,D2,E2,F2,G2,H2
0,seven,ONE,ZERO,ONE,ZERO,ZERO,ONE,ZERO,seven,ONE,ZERO,ONE,ZERO,ZERO,ONE,ZERO
1,one,ZERO,ZERO,ONE,ZERO,ZERO,ONE,ZERO,one,ZERO,ZERO,ONE,ZERO,ZERO,ONE,ZERO
2,four,ZERO,ONE,ONE,ONE,ZERO,ONE,ZERO,four,ZERO,ONE,ONE,ONE,ZERO,ONE,ZERO
3,two,ONE,ONE,ONE,ONE,ONE,ZERO,ZERO,two,ONE,ONE,ONE,ONE,ONE,ZERO,ZERO
4,eight,ZERO,ONE,ONE,ONE,ONE,ONE,ONE,eight,ZERO,ONE,ONE,ONE,ONE,ONE,ONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,five,ONE,ONE,ZERO,ONE,ZERO,ONE,ONE,five,ONE,ONE,ZERO,ONE,ZERO,ONE,ONE
496,seven,ONE,ZERO,ONE,ONE,ONE,ONE,ZERO,seven,ONE,ZERO,ONE,ONE,ONE,ONE,ZERO
497,four,ZERO,ZERO,ONE,ONE,ONE,ZERO,ZERO,four,ZERO,ZERO,ONE,ONE,ONE,ZERO,ZERO
498,zero,ONE,ONE,ONE,ZERO,ZERO,ONE,ZERO,zero,ONE,ONE,ONE,ZERO,ZERO,ONE,ZERO


In [88]:
# Оставляем первые 8 столбцов, поскольку каждый столбец в таблице данных присутствует дважды
df = df.iloc[:, :8]
df

,A,B,C,D,E,F,G,H
0,seven,ONE,ZERO,ONE,ZERO,ZERO,ONE,ZERO
1,one,ZERO,ZERO,ONE,ZERO,ZERO,ONE,ZERO
2,four,ZERO,ONE,ONE,ONE,ZERO,ONE,ZERO
3,two,ONE,ONE,ONE,ONE,ONE,ZERO,ZERO
4,eight,ZERO,ONE,ONE,ONE,ONE,ONE,ONE
...,...,...,...,...,...,...,...,...
495,five,ONE,ONE,ZERO,ONE,ZERO,ONE,ONE
496,seven,ONE,ZERO,ONE,ONE,ONE,ONE,ZERO
497,four,ZERO,ZERO,ONE,ONE,ONE,ZERO,ZERO
498,zero,ONE,ONE,ONE,ZERO,ZERO,ONE,ZERO


In [89]:
# Удаляем пробелы и заменяем слова на цифры
df = df.map(lambda x: x.strip()).replace({'ZERO':0, 'ONE':1, 'zero':0, 'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9})
df

C:\Users\Honor\AppData\Local\Temp\ipykernel_17612\296150356.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.map(lambda x: x.strip()).replace({'ZERO':0, 'ONE':1, 'zero':0, 'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9})


,A,B,C,D,E,F,G,H
0,7,1,0,1,0,0,1,0
1,1,0,0,1,0,0,1,0
2,4,0,1,1,1,0,1,0
3,2,1,1,1,1,1,0,0
4,8,0,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
495,5,1,1,0,1,0,1,1
496,7,1,0,1,1,1,1,0
497,4,0,0,1,1,1,0,0
498,0,1,1,1,0,0,1,0


In [90]:
# Удаляем группирующую переменную - правильная цифра
df_clear = df.drop('A', axis=1)
df_clear

,B,C,D,E,F,G,H
0,1,0,1,0,0,1,0
1,0,0,1,0,0,1,0
2,0,1,1,1,0,1,0
3,1,1,1,1,1,0,0
4,0,1,1,1,1,1,1
...,...,...,...,...,...,...,...
495,1,1,0,1,0,1,1
496,1,0,1,1,1,1,0
497,0,0,1,1,1,0,0
498,1,1,1,0,0,1,0


#### Кластеризация

In [91]:
eps = [0.95, 1, 1.25, 1.5, 1.75, 2, 2.25, 2.5, 2.75, 3, 3.25, 3.5, 3.75, 4]
min_samples = [2, 3, 4, 5, 10, 15, 20]

best_value = {'eps':0, 'min_samples':0, 'silhouette score':-1}

for i in range(len(eps)):
    for j in range(len(min_samples)):

        model = DBSCAN(min_samples=min_samples[j], eps=eps[i]).fit(df_clear)
        labels = model.labels_
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

        if n_clusters == 10:
            silhouette_avg = metrics.silhouette_score(df_clear, labels)
            if silhouette_avg > best_value['silhouette score']:
                best_value.update({'eps': eps[i], 'min_samples': min_samples[j], 'silhouette score': silhouette_avg})

print(best_value)

{'eps': 0.95, 'min_samples': 15, 'silhouette score': np.float64(0.413861592420154)}


In [99]:
# Обучим модель
model = DBSCAN(eps=best_value['eps'], min_samples=best_value['min_samples'])
model.fit(df_clear)

# Добавим в исходные данные столбец 'кластер'
df['cluster'] = model.labels_

# Выведем описательные статистики для каждого кластера
df.groupby('cluster').mean().iloc[:,1:]

,B,C,D,E,F,G,H
cluster,,,,,,,
-1,0.614286,0.504762,0.652381,0.666667,0.457143,0.714286,0.533333
0,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
1,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
2,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000
3,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000
4,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
5,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000
6,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
7,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000


In [100]:
# Интерпретируем, полученнеы результаты с помощью столбца, содержащего правильные цифры
pd.DataFrame(df.groupby(['cluster'])['A'].value_counts())

count
cluster A       
-1      4     27
        3     26
        1     25
        6     23
        7     23
        2     20
        8     19
        5     16
        9     16
        0     15
 0      7     23
        1      1
 1      1     16
        7      2
 2      4     25
        1      1
 3      5     30
        9      3
        6      2
 4      2     22
        0      1
        9      1
 5      9     27
        3      5
        5      5
        8      4
        0      1
 6      8     31
        0      2
        5      1
        6      1
        9      1
 7      6     24
        5      2
        8      2
        2      1
 8      0     23
        3      1
        8      1
 9      3     23
        9      7
        8      1

<br> Кластер 0: цифра 7
<br> Кластер 1: цифра 1
<br> Кластер 2: цифра 4
<br> Кластер 3: цифра 5
<br> Кластер 4: цифра 2
<br> Кластер 5: цифра 9
<br> Кластер 6: цифра 8
<br> Кластер 7: цифра 6
<br> Кластер 8: цифра 0
<br> Кластер 9: цифра 3

In [101]:
df['cluster'] = df.cluster.replace({0:7, 1:1, 2:4, 3:5, 4:2, 5:9, 6:8, 7:6, 8:0, 9:3})
accuracy_score(df.A, df.cluster)

0.488

#### Выводы

В случае иерархической кластеризации при разбиении на 10 кластеров, посчитав для каждого кластера какие цифры в нем встречаются чаще всего при помощи истинных меток, в некоторых кластерах почти одинаково часто встречаются две цифры, например, 5 и 6, 8 и 0, 7 и 1. При увеличении количества кластеров до 12 7 и 1 также объединяются в один кластер. Кроме того, появляются кластеры, интепретация которых затруднена.

При кластеризации методами КMeans и DBSCAN получается выделить 10 однозначно интерпретируемых кластеров. Заменив, метку кластера на ту цифру, представтелей которой в кластере большинство, и сравнив эти метки с истинными, KMeans показал точность выше (74%).